# EDA of the MIMIC dataset

In order to connect to the full database, you will need to install the [pymonetdb](https://pypi.python.org/pypi/pymonetdb) package. You can do this in your terminal with 'conda install pymonetdb' (if you have Anaconda, use this one as it is more stable) or 'pip install pymonetdb'.

Last step before running the notebook: you need access credentials to this database and replace 'fleming' and the specified password by the ones you were provided with.

Access are only available to people that passed the CITI “Data or Specimens Only Research” and completed the different steps to be granted an access. If you wish to request an access, please refer to this [page](https://mimic.physionet.org/gettingstarted/access/)

In [3]:
# Get your credentials
import numpy as np
omop_login = np.load('full_omop_login.npy').item()

# Open a connection to OMOP
from fleming_lib.tools import connect_to_omop
conn = connect_to_omop(login_dict=omop_login)

# Get ready for SQL query processing
import pandas as pd

As a golden rule, DO NOT retrieve entire tables unless it's absolutely necessary!! 

Stats can be computed more efficiently through SQL directly (the database is on a cluster that is most likely way more powerful than your local machine). Downloading of entire tables should only occur for model training and testing ideally.

## Person table
Information related to each patient

In [2]:
pd.read_sql_query('select * from person limit 10;', conn)

,person_id,gender_concept_id,year_of_birth,month_of_birth,day_of_birth,birth_datetime,race_concept_id,ethnicity_concept_id,location_id,provider_id,care_site_id,person_source_value,gender_source_value,gender_source_concept_id,race_source_value,race_source_concept_id,ethnicity_source_value,ethnicity_source_concept_id
0,62063589,8507,2138,7,17,2138-07-17,8515,0,None,None,None,None,M,None,ASIAN,None,None,None
1,62063590,8507,2025,4,11,2025-04-11,8527,0,None,None,None,None,M,None,WHITE,None,None,None
2,62063591,8532,2143,5,12,2143-05-12,8527,0,None,None,None,None,F,None,WHITE,None,None,None
3,62063592,8507,2103,2,2,2103-02-02,8515,0,None,None,None,None,M,None,ASIAN,None,None,None
4,62063593,8532,2109,6,21,2109-06-21,8527,0,None,None,None,None,F,None,WHITE,None,None,None
5,62063594,8532,2121,5,23,2121-05-23,8527,0,None,None,None,None,F,None,WHITE,None,None,None
6,62063595,8507,2117,11,20,2117-11-20,8527,0,None,None,None,None,M,None,WHITE,None,None,None
7,62063596,8507,2108,1,26,2108-01-26,4218674,0,None,None,None,None,M,None,UNKNOWN/NOT SPECIFIED,None,None,None
8,62063597,8532,2103,6,28,2103-06-28,38003599,0,None,None,None,None,F,None,BLACK/AFRICAN AMERICAN,None,None,None
9,62063598,8532,2128,2,22,2128-02-22,8527,0,None,None,None,None,F,None,WHITE,None,None,None


Ethnicity can be aggregated apparently, and we would need the age of patient when they were first processed by the hospital

In [8]:
query = """
select 
    p.race_source_value as ethnicity, p.gender_source_value as gender, 
    avg((v.first_visit_date - p.birth_datetime)/365.25) as avg_age,
    count(0) as count 
from 
    person p
    
    left outer join

    (select 
        person_id, min(visit_start_date) as first_visit_date
    from
        visit_occurrence
    group by 
        person_id) v
    
    on p.person_id = v.person_id
    
group by
    p.race_source_value, p.gender_source_value
order by
    p.race_source_value, p.gender_source_value    
    ;"""
#query = "select datepart(yyyy, birth_datetime) from person limit 5;"
person_age_by_ethnicity_gender = pd.read_sql_query(query, conn)
person_age_by_ethnicity_gender

,ethnicity,gender,avg_age,count
0,AMERICAN INDIAN/ALASKA NATIVE,F,33.774500,20
1,AMERICAN INDIAN/ALASKA NATIVE,M,14.740480,25
2,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,F,60.501000,1
3,AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGN...,M,43.835000,1
4,ASIAN,F,32.566360,572
5,ASIAN,M,32.689960,732
6,ASIAN - ASIAN INDIAN,F,66.120250,20
7,ASIAN - ASIAN INDIAN,M,57.036081,37
8,ASIAN - CAMBODIAN,F,63.049750,4
9,ASIAN - CAMBODIAN,M,47.545167,6


In [6]:
query = """
select 
    p.person_id, (v.visit_start_date - p.birth_datetime)/365.25 as age
from 
    person p
    
    left outer join
    
    visit_occurrence v
    
    on p.person_id = v.person_id
    
limit 10    
    ;"""
#query = "select datepart(yyyy, birth_datetime) from person limit 5;"
temp = pd.read_sql_query(query, conn)
temp

,person_id,age
0,62063589,0.000
1,62063590,76.522
2,62063591,47.841
3,62063592,0.000
4,62063593,65.938
5,62063594,0.000
6,62063595,0.000
7,62063596,41.787
8,62063597,0.000
9,62063598,50.143


## Death table

In [11]:
pd.read_sql_query('select * from death limit 10;', conn)

,person_id,death_date,death_datetime,death_type_concept_id,cause_concept_id,cause_source_value,cause_source_concept_id
0,62063368,2188-11-22,2188-11-22 12:00:00,38003569,None,None,None
1,62063384,2198-02-18,2198-02-18 03:55:00,38003569,None,None,None
2,62063393,2182-07-31,2182-07-31 06:45:00,38003569,None,None,None
3,62063403,2145-03-19,2145-03-19 07:00:00,38003569,None,None,None
4,62063425,2156-08-26,2156-08-26 12:00:00,38003569,None,None,None
5,62063429,2182-02-28,2182-02-28 14:50:00,38003569,None,None,None
6,62063436,2147-11-11,2147-11-11 22:57:00,38003569,None,None,None
7,62063437,2171-06-27,2171-06-27 17:10:00,38003569,None,None,None
8,62063444,2123-07-16,2123-07-16 23:19:00,38003569,None,None,None
9,62063447,2129-12-20,2129-12-20 01:37:00,38003569,None,None,None


## Visit_occurrence table

In [12]:
pd.read_sql_query('select * from visit_occurrence limit 10;', conn)

,visit_occurrence_id,person_id,visit_concept_id,visit_start_date,visit_start_datetime,visit_end_date,visit_end_datetime,visit_type_concept_id,provider_id,care_site_id,visit_source_value,visit_source_concept_id,admitting_concept_id,admitting_source_value,admitting_source_concept_id,discharge_to_concept_id,discharge_to_source_value,discharge_to_source_concept_id,preceding_visit_occurrence_id
0,213,62063589,9201,2138-07-17,2138-07-17 19:04:00,2138-07-21,2138-07-21 15:48:00,44818518,None,67099830,NEWBORN,2001033356,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8536,HOME,2001036536,None
1,214,62063590,262,2101-10-20,2101-10-20 17:09:00,2101-10-31,2101-10-31 13:58:00,44818518,None,67099830,EMERGENCY,2001033355,8870,EMERGENCY ROOM ADMIT,2001031826,8863,SNF,2001036544,None
2,215,62063591,262,2191-03-15,2191-03-15 13:10:00,2191-03-23,2191-03-23 18:41:00,44818518,None,67099830,EMERGENCY,2001033355,8870,EMERGENCY ROOM ADMIT,2001031826,8536,HOME WITH HOME IV PROVIDR,2001036538,None
3,216,62063592,9201,2103-02-02,2103-02-02 04:31:00,2103-02-04,2103-02-04 12:15:00,44818518,None,67099830,NEWBORN,2001033356,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8536,HOME,2001036536,None
4,217,62063593,9201,2175-05-30,2175-05-30 07:15:00,2175-06-15,2175-06-15 16:00:00,44818518,None,67099830,ELECTIVE,2001033354,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8536,HOME HEALTH CARE,2001036537,None
5,218,62063594,9201,2121-05-23,2121-05-23 15:05:00,2121-05-27,2121-05-27 11:57:00,44818518,None,67099830,NEWBORN,2001033356,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8536,HOME,2001036536,None
6,219,62063595,9201,2117-11-20,2117-11-20 10:22:00,2117-11-24,2117-11-24 14:20:00,44818518,None,67099830,NEWBORN,2001033356,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8536,HOME,2001036536,None
7,220,62063596,262,2149-11-09,2149-11-09 11:13:00,2149-11-14,2149-11-14 10:15:00,44818518,None,67099830,EMERGENCY,2001033355,8870,EMERGENCY ROOM ADMIT,2001031826,4216643,DEAD/EXPIRED,2001036535,None
8,221,62063597,9201,2103-06-28,2103-06-28 11:36:00,2103-07-06,2103-07-06 12:10:00,44818518,None,67099830,NEWBORN,2001033356,8940,PHYS REFERRAL/NORMAL DELI,2001031830,8863,SHORT TERM HOSPITAL,2001036543,None
9,222,62063598,262,2178-04-15,2178-04-15 20:46:00,2178-05-11,2178-05-11 19:00:00,44818518,None,67099830,EMERGENCY,2001033355,8870,EMERGENCY ROOM ADMIT,2001031826,8536,HOME HEALTH CARE,2001036537,None


## Measurement table

In [12]:
query = """
select
    *
from
    measurement
limit
    10;"""

pd.read_sql_query(query, conn)

,measurement_id,person_id,measurement_concept_id,measurement_concept_name,measurement_concept_code,measurement_concept_code_system,measurement_date,measurement_datetime,measurement_type_concept_id,measurement_type_concept_name,...,unit_concept_code_system,range_low,range_high,provider_id,visit_occurrence_id,visit_detail_id,measurement_source_value,measurement_source_concept_id,unit_source_value,value_source_value
0,27861030,62105861,3023314,Hematocrit [Volume Fraction] of Blood by Autom...,4544-3,LOINC,2117-09-15,2117-09-15 10:20:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51221,2001031483,%,32.2
1,27851085,62105861,3000330,Specific gravity of Urine by Test strip,5811-5,LOINC,2117-09-13,2117-09-13 08:21:00,2000000009,Labs - Hemato,...,None,None,None,None,47371,None,51498,2001031633,None,1.006
2,27860999,62105861,3003282,Leukocytes [#/volume] in Blood by Manual count,804-5,LOINC,2117-09-14,2117-09-14 01:21:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51301,2001031563,K/uL,10.5
3,27851103,62105861,3014576,Chloride serum/plasma,2075-0,LOINC,2117-09-14,2117-09-14 01:21:00,2000000011,Labs - Chemistry,...,UCUM,None,None,None,47371,None,50902,2001031166,mEq/L,103
4,27831861,62105861,3005897,Protein [Mass/volume] in Urine by Test strip,5804-0,LOINC,2117-09-12,2117-09-12 08:41:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51492,2001031627,mg/dL,100
5,27861037,62105861,3020416,Erythrocytes [#/volume] in Blood by Automated ...,789-8,LOINC,2117-09-15,2117-09-15 10:20:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51279,2001031541,m/uL,3.86
6,27822002,62105861,3024629,Glucose urine dipstick,5792-7,LOINC,2117-09-11,2117-09-11 08:53:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51478,2001031613,mg/dL,300
7,27812669,62105861,3024128,Total Bilirubin serum/plasma,1975-2,LOINC,2117-09-11,2117-09-11 08:22:00,2000000011,Labs - Chemistry,...,UCUM,None,None,None,47371,None,50885,2001031149,mg/dL,0.5
8,27831860,62105861,3022621,pH of Urine by Test strip,5803-2,LOINC,2117-09-12,2117-09-12 08:41:00,2000000009,Labs - Hemato,...,UCUM,None,None,None,47371,None,51491,2001031626,units,6.5
9,27861005,62105861,3000330,Specific gravity of Urine by Test strip,5811-5,LOINC,2117-09-14,2117-09-14 10:08:00,2000000009,Labs - Hemato,...,None,None,None,None,47371,None,51498,2001031633,None,1.006


In [38]:
query = """
select
    m.person_id, m.measurement_datetime, m.measurement_id, m.value_source_value as value, m.unit_source_value as unit, 
    c.concept_name as measurement_type, c1.concept_name as measurement_source
from
    measurement m
    left outer join 
    (select concept_id, concept_name from concept) c
    on m.measurement_type_concept_id = c.concept_id
    left outer join 
    (select concept_id, concept_name from concept) c1
    on m.measurement_source_concept_id = c1.concept_id
where 
    unit_source_value = '%mmHg%'
order by
    person_id, measurement_datetime
limit
    10;"""
bpm = pd.read_sql_query(query, conn)
bpm

,person_id,measurement_datetime,measurement_id,value,unit,measurement_type,measurement_source


## Procedure_occurrence

In [17]:
pd.read_sql_query('select * from procedure_occurrence limit 10;', conn)

,procedure_occurrence_id,person_id,procedure_concept_id,procedure_date,procedure_datetime,procedure_type_concept_id,modifier_concept_id,quantity,provider_id,visit_occurrence_id,visit_detail_id,procedure_source_value,procedure_source_concept_id,qualifier_source_value
0,509041,62105861,4033552,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Critical care services,2514441,None
1,509039,62105861,4033552,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Critical care services,2514441,None
2,509038,62105861,4033552,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Critical care services,2514441,None
3,509037,62105861,4203130,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Hospital inpatient services,2514414,None
4,509043,62105861,4313303,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Hospital inpatient services,2514409,None
5,509036,62105861,4027135,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Hospital inpatient services,2514408,None
6,509042,62105861,4313303,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Hospital inpatient services,2514409,None
7,509040,62105861,4033552,2117-09-11,2117-09-11 11:46:00,257,None,None,None,47371,None,Critical care services,2514441,None
8,342915,62095228,4040551,2150-04-17,2150-04-17 15:34:00,257,None,None,None,47029,None,Diagnostic ultrasound,2211779,None
9,342913,62095228,4033552,2150-04-17,2150-04-17 15:34:00,257,None,None,None,47029,None,Critical care services,2514441,None


## Condition_occurrence table

In [18]:
pd.read_sql_query('select * from condition_occurrence limit 10;', conn)

,condition_occurrence_id,person_id,condition_concept_id,condition_start_date,condition_start_datetime,condition_end_date,condition_end_datetime,condition_type_concept_id,stop_reason,provider_id,visit_occurrence_id,visit_detail_id,condition_source_value,condition_source_concept_id,condition_status_source_value,condition_status_concept_id
0,5622381,62105861,443597,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000194,None,None,47371,None,5853,44827888,None,None
1,5622371,62105861,439770,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000184,None,None,47371,None,25013,44822934,None,None
2,5622382,62105861,373999,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000195,None,None,47371,None,25053,44820684,None,None
3,5622374,62105861,26727,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000187,None,None,47371,None,5780,44819811,None,None
4,5622373,62105861,197320,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000186,None,None,47371,None,5849,44826731,None,None
5,5622379,62105861,200687,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000192,None,None,47371,None,25043,44834549,None,None
6,5622377,62105861,195847,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000190,None,None,47371,None,5363,44822024,None,None
7,5622383,62105861,376683,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000196,None,None,47371,None,36201,44831148,None,None
8,5622386,62105861,0,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,44818709,None,None,47371,None,V1351,44824960,None,None
9,5622385,62105861,134441,2117-09-11,2117-09-11 08:59:00,2117-09-17,2117-09-17 16:45:00,38000198,None,None,47371,None,7078,44831472,None,None


## Observation table

In [19]:
pd.read_sql_query('select * from observation limit 10;', conn)

,observation_id,person_id,observation_concept_id,observation_date,observation_datetime,observation_type_concept_id,value_as_number,value_as_string,value_as_concept_id,qualifier_concept_id,unit_concept_id,provider_id,visit_occurrence_id,visit_detail_id,observation_source_value,observation_source_concept_id,unit_source_value,qualifier_source_value
0,3519795,62105861,4085802,2117-09-11,2117-09-11 00:00:00,38000280,None,20 Gauge Insertion Date,None,None,None,98703,47371,67158807,None,None,None,None
1,3519818,62105861,4085802,2117-09-14,2117-09-14 00:00:00,38000280,None,22 Gauge Insertion Date,None,None,None,98703,47371,67031333,None,None,None,None
2,3519817,62105861,4085802,2117-09-13,2117-09-13 04:00:00,38000280,None,Impaired Skin - Dressing Change #1,None,None,None,98703,47371,67031333,None,None,None,None
3,3519816,62105861,4085802,2117-09-14,2117-09-14 00:00:00,38000280,None,22 Gauge Insertion Date,None,None,None,98703,47371,67031333,None,None,None,None
4,3519790,62105861,4085802,2117-09-11,2117-09-11 00:00:00,38000280,None,20 Gauge Insertion Date,None,None,None,96516,47371,67158807,None,None,None,None
5,3519809,62105861,4085802,2117-09-13,2117-09-13 00:00:00,38000280,None,22 Gauge Insertion Date,None,None,None,98703,47371,67031333,None,None,None,None
6,3519799,62105861,4085802,2117-09-11,2117-09-11 00:00:00,38000280,None,20 Gauge Insertion Date,None,None,None,98703,47371,67158807,None,None,None,None
7,3519801,62105861,4085802,2117-09-13,2117-09-13 01:40:00,38000280,None,22 Gauge Insertion Date,None,None,None,98703,47371,67031333,None,None,None,None
8,3519812,62105861,4085802,2117-09-14,2117-09-14 00:00:00,38000280,None,20 Gauge Insertion Date,None,None,None,100652,47371,67031333,None,None,None,None
9,3519807,62105861,4085802,2117-09-11,2117-09-11 00:00:00,38000280,None,20 Gauge Insertion Date,None,None,None,95926,47371,67158807,None,None,None,None


## Drug_exposure table

In [68]:
pd.read_sql_query('select * from drug_exposure limit 10;', conn)

,drug_exposure_id,person_id,drug_concept_id,drug_exposure_start_date,drug_exposure_start_datetime,drug_exposure_end_date,drug_exposure_end_datetime,verbatim_end_date,drug_type_concept_id,stop_reason,...,route_concept_id,lot_number,provider_id,visit_occurrence_id,visit_detail_id,drug_source_value,drug_source_concept_id,route_source_value,dose_unit_source_value,quantity_source_value
0,65134749,62105861,40168991,2117-09-12,2117-09-12,2117-09-14,2117-09-14,None,38000177,None,...,4112421,None,None,47371,None,00008092355,None,IV,VIAL,1
1,65133109,62105861,0,2117-09-14,2117-09-14,2117-09-15,2117-09-15,None,38000177,None,...,4112421,None,None,47371,None,00338067104,None,IV,BAG,1
2,65133110,62105861,19079524,2117-09-14,2117-09-14,2117-09-15,2117-09-15,None,38000177,None,...,4112421,None,None,47371,None,00338004903,None,IV,mL,500
3,65136366,62105861,19078557,2117-09-11,2117-09-11,2117-09-11,2117-09-11,None,38000177,None,...,4112421,None,None,47371,None,00002821501,None,IV DRIP,mL,1
4,65134747,62105861,0,2117-09-12,2117-09-12,2117-09-13,2117-09-13,None,38000177,None,...,4112421,None,None,47371,None,00338008504,None,IV,mL,1000
5,65136379,62105861,40232756,2117-09-11,2117-09-11,2117-09-17,2117-09-17,None,38000177,None,...,4128794,None,None,47371,None,00406055262,None,PO/NG,TAB,1
6,65134755,62105861,0,2117-09-12,2117-09-12,2117-09-15,2117-09-15,None,38000177,None,...,4112421,None,None,47371,None,0,None,IV,BAG,1
7,65134753,62105861,0,2117-09-12,2117-09-12,2117-09-15,2117-09-15,None,38000177,None,...,4112421,None,None,47371,None,0,None,IV,BAG,1
8,65134751,62105861,0,2117-09-12,2117-09-12,2117-09-15,2117-09-15,None,38000177,None,...,4112421,None,None,47371,None,0,None,IV,BAG,1
9,65134750,62105861,0,2117-09-12,2117-09-12,2117-09-14,2117-09-14,None,38000177,None,...,4112421,None,None,47371,None,0,None,IV,VIAL,1


In [21]:
pd.read_sql_query('select * from note limit 10;', conn)

,note_id,person_id,note_date,note_datetime,note_type_concept_id,note_class_concept_id,note_title,note_text,encoding_concept_id,language_concept_id,provider_id,visit_occurrence_id,note_source_value,visit_detail_id
0,55679581,62105861,2117-09-17,NaT,44814637,0,Report,Admission Date: [**2117-9-11**] ...,0,40639385,NaN,47371,Discharge summary,None
1,56833725,62105861,2117-09-11,2117-09-11 11:12:00,44814641,0,CHEST (PA & LAT),[**2117-9-11**] 11:12 AM\n CHEST (PA & LAT) ...,0,40639385,NaN,47371,Radiology,None
2,56200264,62095228,2150-04-19,2150-04-19 06:28:00,0,0,Physician Resident Progress Note,Chief Complaint:\n 24 Hour Events:\n CALLED ...,0,40639385,97707.0,47029,Physician,None
3,56200473,62095228,2150-04-18,2150-04-18 05:30:00,44814644,0,Nursing Progress Note,"59 year old male with HCV-related cirrhosis, g...",0,40639385,94923.0,47029,Nursing,None
4,56198294,62095228,2150-04-19,2150-04-19 02:35:00,44814644,0,Nursing Progress Note,"59 year old male with HCV-related cirrhosis, g...",0,40639385,94923.0,47029,Nursing,None
5,56200709,62095228,2150-04-18,2150-04-18 02:47:00,0,0,Physician Resident Admission Note,"Chief Complaint: coffee ground emesis, light ...",0,40639385,95486.0,47029,Physician,None
6,55650161,62095228,2150-04-21,NaT,44814637,0,Report,Admission Date: [**2150-4-17**] ...,0,40639385,NaN,47029,Discharge summary,None
7,56200262,62095228,2150-04-19,2150-04-19 06:28:00,0,0,Physician Resident Progress Note,Chief Complaint:\n 24 Hour Events:\n CALLED ...,0,40639385,97707.0,47029,Physician,None
8,55815916,62095228,2150-04-17,NaT,44814642,0,Report,Sinus rhythm\nProlonged QT interval is nonspec...,0,40639385,NaN,47029,ECG,None
9,56198312,62095228,2150-04-18,2150-04-18 02:47:00,0,0,Physician Resident Admission Note,"Chief Complaint: coffee ground emesis, light ...",0,40639385,100299.0,47029,Physician,None


## Note_nlp table

In [22]:
pd.read_sql_query('select * from note_nlp limit 10;', conn)

,note_nlp_id,note_id,section_concept_id,snippet,lexical_variant,note_nlp_concept_id,note_nlp_source_concept_id,nlp_system,nlp_date,nlp_datetime,term_exists,term_temporal,term_modifiers,offset_begin,offset_end,section_source_value,section_source_concept_id
0,118083470,55680405,0,None,Discharge Medications:\n1. aspirin 325 mg Tabl...,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,2936,3584,Discharge Medications:,2001042498
1,118083471,55680405,0,None,Discharge Disposition:\nHome\n\n,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3584,3613,Discharge Disposition:,2001042483
2,118083472,55680405,0,None,Discharge Diagnosis:\nAbdominal Aortic Aneurys...,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3613,3678,Discharge Diagnosis:,2001042489
3,118083473,55680405,0,None,Discharge Condition:\n,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3678,3699,Discharge Condition:,2001042490
4,118083474,55680405,0,None,Mental Status: Clear and coherent.\n,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3699,3734,Mental Status:,2001042516
5,118083475,55680405,2005203326,None,Level of Consciousness: Alert and interactive.\n,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3734,3781,Level of Consciousness:,2001042517
6,118083476,55680405,2005203328,None,Activity Status: Ambulatory - Independent.\n\n\n,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3781,3826,Activity Status:,2001042519
7,118083477,55680405,0,None,Discharge Instructions:\nYou were admitted to ...,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,3826,4195,Discharge Instructions:,2001042495
8,118083478,55680405,0,None,Medications:\n??????\tTake Aspirin 325mg (ente...,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,4195,5283,Medications:,2001042670
9,118083479,55680405,2005203559,None,What activities you can and cannot do:\n??????...,4307844,0,UIMA Section Extractor v1.0,2018-04-14,2018-04-14 23:36:50.552,None,None,None,5283,5981,What activities you can and cannot do:,2001042750


## Cohort_definition table

In [23]:
pd.read_sql_query('select * from cohort_definition limit 10;', conn)

,cohort_definition_id,cohort_definition_name,cohort_definition_description,definition_type_concept_id,cohort_definition_syntax,subject_concept_id,cohort_initiation_date
0,0,No particular cohort,None,0,None,0,2018-04-14
1,1,angus severe,jerome description,0,None,8,2018-04-14
2,2,angus shock,jerome description,0,None,8,2018-04-14
3,3,accp severe,jerome description,0,None,8,2018-04-14
4,4,accp shock,jerome description,0,None,8,2018-04-14
5,5,sepsis3 severe,jerome description,0,None,8,2018-04-14
6,6,sepsis3 shock,jerome description,0,None,8,2018-04-14


## Cohort_attribute table

In [25]:
pd.read_sql_query('select * from cohort_attribute limit 10;', conn)

,cohort_definition_id,cohort_start_date,cohort_end_date,subject_id,attribute_definition_id,value_as_number,value_as_concept_id
0,0,2117-09-11,2117-09-11,67158807,2,0.117072,0
1,0,2117-09-11,2117-09-15,67031333,2,4.256701,0
2,0,2117-09-15,2117-09-17,67031335,2,1.950764,0
3,0,2150-04-17,2150-04-17,67158811,2,0.101181,0
4,0,2150-04-17,2150-04-17,67040892,2,0.010822,0
5,0,2150-04-17,2150-04-17,67040893,2,0.000567,0
6,0,2150-04-17,2150-04-19,67040894,2,1.931088,0
7,0,2150-04-19,2150-04-21,67040895,2,2.141435,0
8,0,2108-04-06,2108-04-06,67158816,2,0.174479,0
9,0,2108-04-06,2108-04-11,66892538,2,4.977639,0


## Attribute_definition table

In [26]:
pd.read_sql_query('select * from attribute_definition limit 10;', conn)

,attribute_definition_id,attribute_name,attribute_description,attribute_type_concept_id,attribute_syntax
0,1,Callout Delay,None,0,None
1,2,Visit Detail Delay,None,0,None
2,3,Visit Occurrence Delay,None,0,None
3,4,duration_hours,ventdurations - duration_hours,0,None
4,5,sofa,sofa - sofa,0,None
5,6,respiration,sofa - respiration,0,None
6,7,coagulation,sofa - coagulation,0,None
7,8,liver,sofa - liver,0,None
8,9,cardiovascular,sofa - cardiovascular,0,None
9,10,cns,sofa - cns,0,None


## Care_site table

In [27]:
pd.read_sql_query('select * from care_site limit 10;', conn)

,care_site_id,care_site_name,place_of_service_concept_id,location_id,care_site_source_value,place_of_service_source_value
0,67099738,CCU ward #7,4149943,None,CCU,Coronary care unit
1,67099739,CCU ward #37,4149943,None,CCU,Coronary care unit
2,67099740,CCU ward #57,4149943,None,CCU,Coronary care unit
3,67099741,CSRU ward #33,4138949,None,CSRU,Cardiac surgery recovery unit
4,67099742,CSRU ward #12,4138949,None,CSRU,Cardiac surgery recovery unit
5,67099743,CSRU ward #14,4138949,None,CSRU,Cardiac surgery recovery unit
6,67099744,CSRU ward #23,4138949,None,CSRU,Cardiac surgery recovery unit
7,67099745,CSRU ward #15,4138949,None,CSRU,Cardiac surgery recovery unit
8,67099746,CSRU ward #47,4138949,None,CSRU,Cardiac surgery recovery unit
9,67099747,MICU ward #52,40481392,None,MICU,Medical intensive care unit


## Concept table

In [25]:
pd.read_sql_query('select * from concept limit 10;', conn)

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,2103359,"Excision of rib, partial",Procedure,CPT4,CPT4,S,21600,1970-01-01,2099-12-31,None
1,2103847,"Excision, radial head",Procedure,CPT4,CPT4,S,24130,1970-01-01,2099-12-31,None
2,2104737,"Tenotomy, adductor of hip, percutaneous (separ...",Procedure,CPT4,CPT4,S,27000,1970-01-01,2099-12-31,None
3,2105444,"Arthrodesis, ankle, open",Procedure,CPT4,CPT4,S,27870,1970-01-01,2099-12-31,None
4,2105876,Strapping; hand or finger,Procedure,CPT4,CPT4,S,29280,1970-01-01,2099-12-31,None
5,2106077,"Arthroscopy, subtalar joint, surgical; with de...",Procedure,CPT4,CPT4,S,29906,2008-04-01,2099-12-31,None
6,2106527,"Laryngoscopy, flexible; diagnostic",Procedure,CPT4,CPT4,S,31575,1970-01-01,2099-12-31,None
7,42740408,Speculoscopy,Procedure,CPT4,CPT4,None,0031T,2003-03-31,2009-04-06,D
8,42740457,"Starch granules, feces",Procedure,CPT4,CPT4,None,89225,2004-03-31,2011-05-05,D
9,2108119,"Transfusion, blood or blood components",Procedure,CPT4,CPT4,S,36430,1970-01-01,2099-12-31,None


In [26]:
query = """
select 
    *
from
    concept
where
    concept_name LIKE '%Chloride%'
and 
    domain_id = 'Measurement'
    ;"""

search = pd.read_sql_query(query, conn)
search

,concept_id,concept_name,domain_id,vocabulary_id,concept_class_id,standard_concept,concept_code,valid_start_date,valid_end_date,invalid_reason
0,2001019318,label:[ABG Chloride]dbsource:[carevue]linksto:...,Measurement,MIMIC d_items,ABG's,None,4193,1979-01-01,2099-01-01,None
1,2001019976,label:[Chloride (100-112)]dbsource:[carevue]li...,Measurement,MIMIC d_items,Chemistry,None,788,1979-01-01,2099-01-01,None
2,2001020851,label:[Calcium Chloride]dbsource:[carevue]link...,Measurement,MIMIC d_items,None,None,5753,1979-01-01,2099-01-01,None
3,2001021134,label:[Chloride (100-112)]dbsource:[carevue]l...,Measurement,MIMIC d_items,Chemistry,None,3747,1979-01-01,2099-01-01,None
4,2001021240,label:[Chloride]dbsource:[carevue]linksto:[cha...,Measurement,MIMIC d_items,Chemistry,None,1523,1979-01-01,2099-01-01,None
5,2001022289,label:[Potassium Chloride]dbsource:[carevue]li...,Measurement,MIMIC d_items,None,None,3578,1979-01-01,2099-01-01,None
6,2001022320,label:[Sodium Chloride]dbsource:[carevue]links...,Measurement,MIMIC d_items,None,None,3624,1979-01-01,2099-01-01,None
7,2001024119,label:[Sodium Chloride]dbsource:[carevue]links...,Measurement,MIMIC d_items,None,None,30296,1979-01-01,2099-01-01,None
8,2001024120,label:[Potassium Chloride]dbsource:[carevue]li...,Measurement,MIMIC d_items,None,None,30297,1979-01-01,2099-01-01,None
9,2001024123,label:[Calcium Chloride 10%]dbsource:[carevue]...,Measurement,MIMIC d_items,None,None,30300,1979-01-01,2099-01-01,None


# Filter your data

## Limitations
Starting with the limitations, we need to make sure our dataset only integrates people that do not originally depend on medical critical assistance equipment

In [3]:
query = """
select 
    concept_id, concept_code, domain_id, concept_name
from
    concept
where
    concept_id in (2001018843, 2001030812)
    ;"""

limitations = pd.read_sql_query(query, conn)
limitations

,concept_id,concept_code,domain_id,concept_name
0,2001018843,128,Measurement,label:[Code Status]dbsource:[carevue]linksto:[...
1,2001030812,223758,Measurement,label:[Code Status]dbsource:[metavision]linkst...


People that were once not limited

In [15]:
query = """
select 
    count(distinct person_id)
from
    measurement
where
    measurement_type_concept_id = 44818701
    and measurement_concept_id = 4127294
    and value_source_value in ('Full code', 'Full Code')
    ;"""

temp = pd.read_sql_query(query, conn)
temp

,L3
0,30833


People that have been limited

In [2]:
query = """
select 
    person_id, min(measurement_datetime)
from
    measurement
where
    measurement_type_concept_id = 44818701
    and measurement_concept_id = 4127294
    and value_source_value in ('Full code', 'Full Code')
group by
    person_id
    ;"""

temp = pd.read_sql_query(query, conn)
temp

Index(['measurement_id', 'person_id', 'measurement_concept_id',
       'measurement_concept_name', 'measurement_concept_code',
       'measurement_concept_code_system', 'measurement_date',
       'measurement_datetime', 'measurement_type_concept_id',
       'measurement_type_concept_name', 'operator_concept_id',
       'operator_concept_name', 'value_as_number', 'value_as_concept_id',
       'unit_concept_id', 'unit_concept_name', 'unit_concept_code',
       'unit_concept_code_system', 'range_low', 'range_high', 'provider_id',
       'visit_occurrence_id', 'visit_detail_id', 'measurement_source_value',
       'measurement_source_concept_id', 'unit_source_value',
       'value_source_value'],
      dtype='object')

In [4]:
query = """

select
    person_id, min(measurement_datetime) as first_limitation_datetime
from
    measurement
where
    measurement_id not in 
                            (select 
                                measurement_id
                            from
                                measurement
                            where
                                measurement_type_concept_id = 44818701
                                and measurement_concept_id = 4127294
                                and value_source_value in ('Full code', 'Full Code'))
group by
    person_id
limit 10
    ;"""

temp = pd.read_sql_query(query, conn)
temp

,person_id,first_limitation_datetime
0,62105861,2117-09-11 08:22:00
1,62095228,2150-04-17 12:30:00
2,62073122,2108-04-06 11:30:00
3,62102210,2176-08-08 23:25:00
4,62080413,2145-03-30 21:15:00
5,62064070,2155-06-21 01:26:00
6,62085950,2109-08-17 04:30:00
7,62088661,2109-12-10 10:25:00
8,62106943,2177-08-29 03:35:00
9,62096723,2177-03-12 13:02:00


## Age and ICU
Moving to the next filters:
- Age >= 15
- Processed only by intensive care unit

In [13]:
query = """
select
    count(distinct p.person_id)
from 
    person p
    
    inner join

    (select 
        person_id, min(visit_start_date) as first_visit_date
    from
        visit_occurrence
    group by 
        person_id) v
    
    on p.person_id = v.person_id
    
where
    (v.first_visit_date - p.birth_datetime) >= 15 * 365.25
    and p.person_id in (select
                            person_id
                        from
                            visit_detail
                        where
                            visit_detail_concept_id = 581382
                            and visit_type_concept_id = 2000000006)
    ;"""

temp = pd.read_sql_query(query, conn)
temp

,L11
0,38604


## Fleming filter

In [16]:
query = """
select
    count(distinct p.person_id)
from 
    person p
    
    inner join

    (select 
        person_id, min(visit_start_date) as first_visit_date
    from
        visit_occurrence
    group by 
        person_id) v
    
    on p.person_id = v.person_id
    
where
    (v.first_visit_date - p.birth_datetime) >= 15 * 365.25
    and p.person_id not in (select 
                                distinct person_id
                            from
                                measurement
                            where
                                not(measurement_type_concept_id = 44818701
                                and measurement_concept_id = 4127294
                                and value_source_value in ('Full code', 'Full Code')))
    ;"""

temp = pd.read_sql_query(query, conn)
temp

,L25
0,0
